In [148]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import numpy as np

In [164]:
train = pd.read_csv('train.tsv',sep='\t')
train.head(3)

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1


In [165]:
test = pd.read_csv('test.tsv',sep='\t')
test.head(3)

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio
0,http://www.lynnskitchenadventures.com/2009/04/...,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc...",recreation,0.443906,2.558140,0.389706,0.257353,0.044118,0.022059,...,0.199438,1,1,15,0,5643,136,3,0.242647,0.080597
1,http://lolpics.se/18552-stun-grenade-ar,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f...",culture_politics,0.135844,3.771429,0.461538,0.205128,0.051282,0.000000,...,0.080000,?,1,62,0,382,39,2,0.128205,0.176471
2,http://www.xcelerationfitness.com/treadmills.html,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st...",?,?,2.269565,0.495726,0.384615,0.170940,0.170940,...,10.000000,?,1,42,0,2420,117,1,0.581197,0.125000


In [151]:
train_ac = train.alchemy_category.unique()
test_ac = test.alchemy_category.unique()
print 'train_ac',train_ac.shape
print 'test_ac',test_ac.shape

train_ac (14,)
test_ac (12,)


In [12]:
# 样本均衡
train.label.value_counts()

1    3796
0    3599
Name: label, dtype: int64

In [179]:
# 由于分类特征字段数不一样多，所以先合并train_test,再get_dummies
data = pd.concat((train,test),axis=0)
# 网址、网址ID、样板文字对预测影响不大，所以移除
submit = test.urlid
data = data.drop(['url','urlid','boilerplate'],axis = 1)
print 'train', train.shape
print 'test', test.shape
print 'data', data.shape
data.head(3)

train (7395, 27)
test (3171, 26)
data (10566, 24)


,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,frameTagRatio,...,is_news,label,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio
0,business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0.090774,...,1,0.0,1,24,0,5424,170,8,0.152941,0.079130
1,recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0.098707,...,1,1.0,1,40,0,4973,187,9,0.181818,0.125448
2,health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0.072448,...,1,1.0,1,55,0,2240,258,11,0.166667,0.057613


In [180]:
# 4-25是数值特征，缺失值用0替换
def preprocess(df):
    hh = df.loc[:,'alchemy_category_score':'spelling_errors_ratio']
    hh.replace(to_replace='?',value=0,inplace=True)
    hh.fillna(0,inplace=True)
    hh = hh.astype(float)
    df.loc[:,'alchemy_category_score':'spelling_errors_ratio'] = hh
    df.alchemy_category.replace(to_replace='?',value='Unknown',inplace=True)
    df = pd.concat((df,pd.get_dummies(df.alchemy_category)),axis=1)
    df.drop('alchemy_category',axis=1,inplace=True)
    return df

In [181]:
data = preprocess(data)
print 'data', data.shape
data.head(3)

data (10566, 37)


,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,frameTagRatio,framebased,...,culture_politics,gaming,health,law_crime,recreation,religion,science_technology,sports,unknown,weather
0,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0.090774,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0.098707,0.0,...,0,0,0,0,1,0,0,0,0,0
2,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0.072448,0.0,...,0,0,1,0,0,0,0,0,0,0


In [200]:
train1 = data.iloc[:7395,:]
test1 = data.iloc[7395:,:]
print 'train', train1.shape
print 'test', test1.shape

train (7395, 37)
test (3171, 37)


In [202]:
X = train1.iloc[:,:-1]
y = train1.iloc[:,-1]

In [204]:
clf = DecisionTreeClassifier(criterion="entropy",max_depth=10, max_leaf_nodes=10) 
clf.fit(X,y)
y_predict = clf.predict(test1.iloc[:,:-1])
y_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [210]:
submit_df = pd.DataFrame({'urlid':submit,'label':y_predict}, columns=['urlid','label'])
submit_df.to_csv('submit.csv',index=None)